In [4]:
import lxml.etree as etree
import re
from datetime import datetime

In [5]:
tree = etree.parse('../data/19wahlperiode/19064-data.xml')
#etree.strip_tags(tree, 'inline')
root = tree.getroot()

In [3]:
for child in tree.xpath('sitzungsverlauf'):
    for grandchild in child:
        print(grandchild)

<Element sitzungsbeginn at 0x7f0fcc839148>
<Element tagesordnungspunkt at 0x7f0fcc839188>
<Element tagesordnungspunkt at 0x7f0fcc839208>
<Element tagesordnungspunkt at 0x7f0fcc839148>
<Element tagesordnungspunkt at 0x7f0fcc820f88>
<Element sitzungsende at 0x7f0fcc820f08>


### Metadata

In [4]:
# wp under wahlperiode, session under sitzung-nr, date in sitzung-datum, all in dbtplenarprotokoll
for textinfo in tree.xpath("//dbtplenarprotokoll"):
    wp = textinfo.get("wahlperiode")
    session = textinfo.get("sitzung-nr")
    date = textinfo.get("sitzung-datum")

print("Parliament {}/{}".format(wp, session))
print(date) # date as text or datetime object?, currently d-m-Y

Parliament 19/64
21.11.2018


In [5]:
tree.xpath("//dbtplenarprotokoll/@sitzung-datum")

['21.11.2018']

In [45]:
wp = tree.xpath('vorspann/kopfdaten/plenarprotokoll-nummer/wahlperiode/text()')
session = tree.xpath('vorspann/kopfdaten/plenarprotokoll-nummer/sitzungsnr/text()')
for datum in tree.xpath('vorspann/kopfdaten/datum'):
    date = datum.get("date")
print(date)

datetime.strptime(date, '%d.%m.%Y')

24.10.2017


datetime.datetime(2017, 10, 24, 0, 0)

In [ ]:
date = tree.xpath('vorspann/kopfdaten/datum/')

### Texts

In [6]:
divs = tree.findall("//rede")

In [7]:
divs_to = tree.findall("//tagesordnungspunkt")

In [61]:
count = 0
for div in divs_to:
    for uts in div.iter('rede'):
        count += 1
        #print(uts.tag)
print(count)

77


In [18]:
# extract tops 
for div in divs_to:
    for top in div.xpath('child::p'):
        if top.get('klasse') == "T_NaS":
            print(top.text)

hier: Einzelplan 04  Bundeskanzlerin und Bundeskanzleramt
hier: Einzelplan 05 Auswärtiges Amt
hier: Einzelplan 14 Bundesministerium der Verteidigung
hier: Einzelplan 23 Bundesministerium für wirtschaftliche Zusammenarbeit und Entwicklung


In [20]:
# getting to utterances from divs_to
for div in divs_to:
    for sp in div:
        if sp.tag == "rede":
            for uts in sp:
                print(uts.tag)

p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
name
p
p
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
name
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
p
kommentar
p
kommentar
p
p
kommentar
p
p
p
p
kommentar
p
p
kommentar
p
kommentar
p
p
kommentar
name
p
p
p
name
p
p
p
p
p
kommentar
p
kommentar
p
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
p
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p


p
kommentar
p
kommentar
p
p
kommentar
p
p
kommentar
p
p
p
kommentar
name
p
kommentar
p
p
p
kommentar
p
kommentar
p
kommentar
p
p
p
p
kommentar
p
kommentar
p
p
p
p
p
kommentar
name
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
name
p
kommentar
p
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
p
p
kommentar
p
p
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar
name
p
kommentar
p
p
p
p
kommentar
p
p
p
kommentar
p
p
p
p
kommentar
p
p
p
p
kommentar
p
kommentar
p
p
p
p
p
kommentar
p
p
kommentar
name
p
kommentar
p
p
p
p
p
kommentar
name
p
p
p
p
kommentar
name
p
kommentar
p
p
p
p
p
p
kommentar
p
kommentar
p
p
kommentar
p
kommentar
p
kommentar
p
kommentar
p
p
kommentar

In [27]:
for div in divs: # to in tagesordnungspunkt list
    rid = div.get('id')
    print(rid)

ID19100100
ID19100200
ID19100300
ID19100400
ID19100500
ID19100600
ID19100700
ID19100800
ID19100900
ID19101000
ID19101100
ID19101200
ID19101300
ID19101400
ID19101500
ID19101600
ID19101700
ID19101800
ID19101900


In [88]:
# correct structure 
for div in divs:
    for uts in div:
        klasse = uts.get('klasse')
        print(uts.tag, klasse)
    break

p redner
p J_1
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p O
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p O
name None
p J_1
p redner
p J_1
kommentar None
p O
kommentar None
p J
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p O
kommentar None
p O
kommentar None
p J
kommentar None
p J
kommentar None
p O
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O
kommentar None
p O
kommentar None
p O
p J
kommentar None
p O
kommentar None
p O

In [8]:
# using xpath
# testing removing repeating utterances
for div in divs: # list of <rede>
    for uts in div: # child elements of <rede>
        name_count = 0
        if uts.tag == "p" and uts.get("klasse") == "redner":
            ### finding name 
            for redner in uts: # <redner>
                vorname = redner.xpath('name/vorname/text()')
                nachname = redner.xpath('name/nachname/text()')
                fullname = vorname + nachname
                if len(fullname) > 1:
                    fullname = fullname[0] + ' ' + fullname[1]
                if fullname != []:
                    print("Speaker:", fullname)
            
            ### subsequent speech text
            for j in uts.xpath('following-sibling::p[count(preceding-sibling::name) = 0]'):
                print(j.tag, "-", j.text)
            print("--------")    
                
        elif uts.tag == "name":
            name_count += 1
            if uts.text:
                print(uts.tag, uts.text)
                for k in uts.xpath('following-sibling::p'):
                    #[count(preceding-sibling::name) = 1]
                    if k.get('klasse') == 'redner':
                        pass
                    else: 
                        print("N", k.tag, "-", k.text)
                        
        elif uts.tag == "kommentar":
            print(uts.tag, "-" ,uts.text)
                        
    print("--------")                

Speaker: Alice Weidel
p - Sehr geehrter Herr Präsident! Sehr geehrte Damen und Herren! Liebe Kollegen! Zum zweiten Mal in diesem Jahr beraten wir über den Entwurf eines Bundeshaushalts. Dazugelernt hat Ihre Regierung seither nichts, Frau Bundeskanzlerin.
p - Sie betreiben weiterhin eine Politik der Spaltung und der Unvernunft, eine Politik des Ausgabenwahns und der falschen Prioritäten, eine Politik, die den noch vorhandenen Wohlstand von heute bedenkenlos mit vollen Händen ausgibt und verschleudert, ohne an morgen zu denken.
p - Geändert hat sich aber die ökonomische Grundlage, auf der Sie diesen Ausverkauf betreiben. Das wirtschaftliche Fundament bröckelt. Ihre Politik ignoriert konsequent die ökonomische Vernunft. Sie setzt die Interessen des eigenen Landes und der eigenen Bürger an die letzte Stelle. Dafür rennen Sie ideologischen Weltbeglückungsfantasien hinterher, und das hält auch die stärkste Volkswirtschaft auf Dauer nicht ohne Schaden aus.
p - Die Automobilindustrie, ein Eckp

p - Herr Präsident! Liebe Kolleginnen und Kollegen! Sehr geehrte Frau Kanzlerin! Deutschland und die Welt um uns herum verändern sich rasend schnell: durch die Globalisierung, durch den digitalen Wandel, durch die Klimakrise, die mit diesem Dürresommer endgültig für alle sichtbar bei uns angekommen ist.
p - Die Frage ist: Gestalten wir diese Entwicklung, oder lassen wir uns von ihr überrollen? All dies fordert alle und uns im Parlament ganz besonders. Viele dieser Herausforderungen können wir nicht alleine lösen, sondern nur in einer starken Europäischen Union mit internationaler Zusammenarbeit.
p - Wie sichern wir den sozialen Frieden? Wie verhindern wir die Klimakatastrophe? Wie stärken wir ein geeintes Europa? Auf all diese Fragen muss eine verantwortungsvolle Regierung Antworten suchen und finden. Aber wie sieht die Realität aus? Wir erleben eine Koalition, die als große Selbsthilfegruppe vor allem mit sich selbst beschäftigt ist und schon lange nicht mehr mit den Sorgen und den Nö

p - Ja, die Regierung ist unkreativ. Sie verschwendet unser Geld für sinnlose Projekte und investiert viel zu wenig in unsere Zukunft. Was aber verstörend an dieser Debatte war, ist: Sie verweisen ständig auf die EU-Ebene. Das wurde wirklich deutlich. Viele Redner sprachen von einer Lösung auf einer supranationalen Ebene, Macron fordert ein Euro-Zonenbudget, Sie, Frau Merkel, gewähren das mal eben leichtfertig. Aber wofür soll es denn dienen? Das ist ein Topf, in den wieder die Gelder unserer Bürger abfließen. Neben den bereits gut 30 Milliarden Euro im Jahr, die wir zahlen, geht immer mehr an irgendeine supranationale Ebene.
p - Wissen Sie eigentlich, warum uns Estland und Lettland in der Digitalisierung überflügeln? Während Sie auf Fonds aus Brüssel warten, nehmen die ihr Schicksal mal eben selbst in die Hand. Sie sollten mal begreifen: Im europäischen Kontext national zu handeln, bedeutet nicht, gegen jemanden, sondern für das eigene Land und damit indirekt für alle zu handeln. Dies

Speaker: Heike Hänsel
p - Danke schön, Herr Präsident. – Herr Außenminister Maas, ich will nachhaken, weil Sie am Ende Ihrer Rede zweimal erwähnt haben, dass wir eine nukleare Auseinandersetzung vermeiden müssen, und den INF-Vertrag genannt haben. Meine konkrete Frage wäre: Schließen Sie für die Bundesregierung aus, dass neue landgestützte, atomar bestückbare Mittelstreckenraketen vonseiten der USA in Deutschland stationiert werden? Schließt die Bundesregierung, schließen Sie das aus?
--------
name Vizepräsident Dr. Hans-Peter Friedrich:
N p - Herr Minister, kann gleich die zweite Zwischenfrage gestellt werden? – Dann können Sie im Zusammenhang antworten.
--------
Speaker: Frank Müller-Rosentritt
p - Sehr geehrter Herr Bundesminister, mein Kollege Bijan Djir-Sarai hat Sie in seiner Rede gefragt, wie Sie das Abstimmungsverhalten der Bundesregierung bei der UN-Vollversammlung in der letzten Woche erklären. Denn das wirft für uns große Fragen auf. Ich würde mich freuen, wenn Sie diese Fra

Speaker: Tobias Lindner
p - Vielen Dank, Herr Präsident, dass Sie diese Kurzintervention zulassen. – Frau Ministerin, Sie haben mich direkt angesprochen mit Hinweis auf Ihre Antwort auf unsere Kleine Anfrage. Ich möchte nicht den Eindruck entstehen lassen, dass wir hier mit unsauberen Methoden, ganz gleich wo oder wer, arbeiten. Sie haben uns geantwortet, dass Sie Unvollständigkeiten und Doppelungen aufgrund der Kürze der Zeit bei der Beantwortung der Anfrage nicht ausschließen können.
p - Ich finde, die Bürgerinnen und Bürger, die diese Debatte verfolgen, sollten wissen, dass Kürze der Zeit bei der Beantwortung 14 Tage bedeutet. Der Punkt, den ich und meine Fraktion kritisieren, ist: Wenn es Ihnen als Ministerin nicht möglich ist, binnen 14 Tagen einen Überblick zu erlangen, wo überall in Ihrem Haus Beratungsfirmen, sei es beratend oder unterstützend oder für sonst etwas, tätig sind, dann haben Sie nach unserer politischen Bewertung den Überblick über diese Vorgänge verloren bzw. nich

N p - Dritter Schwerpunkt ist: Ich setze erneut das Thema „gerechter und fairer Handel in globalen Lieferketten“ auf die Tagesordnung; denn der Markt braucht Moral, er braucht Regeln, und er braucht Grenzen. Wir brauchen eine neue Wachstumsphilosophie. Wir können, wollen und werden die Welt ein Stück humaner, gerechter und friedlicher machen. Es gibt nichts Gutes, außer man tut es.
N p - Herzlichen Dank.
N p - Vielen Dank, Herr Minister. – Nun darf sich die Unionsfraktion mit der Frage beschäftigen, wem ich die Minute abziehen soll, die der Minister die angemeldete Redezeit überschritten hat.
N p - Als Nächstes spricht zu uns der Kollege Markus Frohnmaier, AfD-Fraktion.
Speaker: Gerd Müller
--------
name Vizepräsident Wolfgang Kubicki:
N p - Die Fraktion Die Linke.
N p - Ja, gerne.
N p - Danke schön, Herr Präsident! – Herr Minister Müller, Sie haben sich ja gerade auf den globalen Vertrag für Migration bezogen und gesagt, er werde Standards schaffen. Gleichzeitig betont die Bundesregie

In [13]:
#### use lists instead of xpath/iter
#### this works! 
# testing removing repeating utterances
speech_list = ['J','O','J_1']
for div in divs: # list of <rede>
    for uts in div: # child elements of <rede>
        if uts.tag == "p" and uts.get("klasse") == "redner" or uts.tag =="name":
            if uts.tag == "p":
                ### finding name 
                for redner in uts: # <redner>
                    vorname = redner.xpath('name/vorname/text()')
                    nachname = redner.xpath('name/nachname/text()')
                    fullname = vorname + nachname
                    if len(fullname) > 1:
                        fullname = fullname[0] + ' ' + fullname[1]
                    if fullname != []:
                        print("Speaker:", fullname)
            
            elif uts.tag == "name":
                print("Speaker:", uts.text)
                
        elif uts.tag == "p" and uts.get("klasse") in speech_list:
            print(uts.tag, "-", uts.text)
        
        elif uts.tag == "kommentar":
            print(uts.tag, "-", uts.text)
        

Speaker: Alice Weidel
p - Sehr geehrter Herr Präsident! Sehr geehrte Damen und Herren! Liebe Kollegen! Zum zweiten Mal in diesem Jahr beraten wir über den Entwurf eines Bundeshaushalts. Dazugelernt hat Ihre Regierung seither nichts, Frau Bundeskanzlerin.
kommentar - (Beifall bei der AfD – Johannes Kahrs [SPD]: Sie sollten sagen, wer Ihre Spender sind!)
p - Sie betreiben weiterhin eine Politik der Spaltung und der Unvernunft, eine Politik des Ausgabenwahns und der falschen Prioritäten, eine Politik, die den noch vorhandenen Wohlstand von heute bedenkenlos mit vollen Händen ausgibt und verschleudert, ohne an morgen zu denken.
kommentar - (Beifall bei der AfD)
p - Geändert hat sich aber die ökonomische Grundlage, auf der Sie diesen Ausverkauf betreiben. Das wirtschaftliche Fundament bröckelt. Ihre Politik ignoriert konsequent die ökonomische Vernunft. Sie setzt die Interessen des eigenen Landes und der eigenen Bürger an die letzte Stelle. Dafür rennen Sie ideologischen Weltbeglückungsfant

kommentar - (Beifall bei der AfD)
p - In einem Zeitungsinterview erklärte Alexis Spire, Forschungsdirektor am Centre national de la recherche scientifique: Einer der Gründe für die Proteste sei, dass Steuerzahler am unteren Ende der sozialen Pyramide keine Gegenleistung mehr zu sehen bekommen für das, was sie einzahlen. Es sei nicht überraschend, dass die Bewegung in ländlichen Gebieten und mittelgroßen Städten entstanden ist. Diese Territorien litten seit langem unter dem allgemeinen Niedergang und der Verschlechterung öffentlicher Dienstleistungen. – Ist das so weit weg von unseren eigenen Problemen in abgehängten Regionen, meine Damen und Herren?
p - Bevor er im Namen Frankreichs seine betörende Liebeserklärung an Deutschland aussprach, sagte Präsident Macron – ich zitiere –:
p - Nun, meine Damen und Herren, es wäre nicht die erste aus finanziellen Gründen arrangierte Ehe, die öffentlich als Liebesheirat verkauft wird.
kommentar - (Beifall bei der AfD)
p - Was uns betrifft: Wir beke

p - Wie trifft man sachlich fundierte, qualifizierte Entscheidungen im Außen, wie beim Migrationspakt und vor allem dessen Kosten, wenn man sein eigenes Haus nicht im Griff hat, Herr Maas?
kommentar - (Beifall bei der AfD)
p - Ich wiederhole: Allen Parteien ist dieser Bericht des Bundesrechnungshofs bekannt, und nur wir, die AfD, thematisieren das als Partei ernsthaft in der Öffentlichkeit. Die AfD fordert, die Mittel für humanitäre Hilfe und Krisenprävention so lange mit sofortiger Wirkung einzufrieren,
kommentar - (Frank Schwabe [SPD]: Was ist mit den Millionen von Menschen, die darunter leiden werden?  Weitere Zurufe von der SPD)
p - bis die ordnungsgemäße und wirtschaftliche Vergabe dieser Zuwendungen gesichert ist. Denn nur so besteht die Chance, dass das Auswärtige Amt deutsche Steuergelder transparent, nachvollziehbar und vor allem sinn- und zweckgemäß einsetzt.
p - Danke schön.
kommentar - (Beifall bei der AfD)
Speaker: Vizepräsident Thomas Oppermann:
p - Das Wort hat die Kolle

kommentar - (Beifall bei Abgeordneten der SPD sowie des Abg. Michael Brand [Fulda] [CDU/CSU])
p - und zwar auch keine, die schon genehmigt worden sind. Ich glaube, das sollten wir im Deutschen Bundestag gemeinsam würdigen.
p - Vielen herzlichen Dank.
kommentar - (Beifall bei der SPD und der CDU/CSU)
Speaker: Vizepräsident Dr. Hans-Peter Friedrich:
p - Der nächste Redner ist der Kollege Dr. Norbert Röttgen, CDU/CSU-Fraktion.
kommentar - (Beifall bei der CDU/CSU)
Speaker: Norbert Röttgen
p - Sehr geehrter Herr Präsident! Liebe Kolleginnen und Kollegen! Die Bundeskanzlerin hat heute Morgen in ihrer Rede gesagt, wir hätten eine manchmal auch sehr nervöse Zeit. Das halte ich für eine bemerkenswerte Beschreibung
kommentar - (Armin-Paulus Hampel [AfD]: Wir auch!)
p - sozusagen der psychischen Verfassung der Welt. Der Bundesaußenminister hat für Deutschland die Frage gestellt: Was liegt eigentlich noch in unserer Gestaltungsmacht? Die Bundeskanzlerin hat übrigens ihrer Feststellung den Hinweis

kommentar - (Dr. Bernd Baumann [AfD]: Das erreichen Sie beides!)
p - Ich glaube, für unser Land ist die Variante, die wir gewählt haben, die wesentlich bessere.
kommentar - (Beifall bei der CDU/CSU)
p - Ja, wir haben einen soliden Haushalt vorgelegt. Wir haben in der Bereinigungssitzung bis freitagmorgens um 5.09 Uhr gerungen.
kommentar - (Zurufe von Abgeordneten der CDU/CSU: Hey!)
p - – Ja, es ist nun mal die Gegebenheit in dieser Bereinigungssitzung, dass sich alle Ministerien dem Haushaltsausschuss noch einmal stellen müssen. Wir haben aber natürlich auch die Anträge aller Fraktionen im Verteidigungsausschuss sehr intensiv beraten, nicht so wie es in einer anderslautenden Pressemitteilung aus dem rechten Teil des Saales verlautbart wurde.
p - Ich muss ganz ehrlich sagen: Zu Recht haben sich die Berichterstatter untereinander für die gute Zusammenarbeit bedankt. Die Ministerin hat sich bedankt. Aber, lieber Reinhard Brandl, was du in diesem Haushalt, deinem zweiten Verteidigungshaush

In [31]:
#### use lists instead of xpath/iter
#### use divs_to instead of divs to extract agenda points  
#### testing removing repeating utterances
speech_list = ['J','O','J_1']
for div in divs_to: # list of <tagesordnungspunkt>
    # finding agenda item
    for top in div.xpath('child::p'):
        if top.get('klasse') == "T_NaS":
            print("Agenda Item:", top.text)
    
    for sp in div:
        if sp.tag == "rede": # finding <rede> in <tagesordnungspunkt>
            for uts in sp: # child elements of <rede>
                if uts.tag == "p" and uts.get("klasse") == "redner" or uts.tag =="name":
                    if uts.tag == "p":
                        ### finding name 
                        for redner in uts: # <redner>
                            vorname = redner.xpath('name/vorname/text()')
                            nachname = redner.xpath('name/nachname/text()')
                            fullname = vorname + nachname
                            if len(fullname) > 1:
                                fullname = fullname[0] + ' ' + fullname[1]
                            if fullname != []:
                                print("Speaker:", fullname)

                    elif uts.tag == "name":
                        print("Speaker:", uts.text.strip(':'))

                elif uts.tag == "p" and uts.get("klasse") in speech_list:
                    print(uts.tag, "-", uts.text)

                elif uts.tag == "kommentar":
                    print(uts.tag, "-", uts.text)

Agenda Item: hier: Einzelplan 04  Bundeskanzlerin und Bundeskanzleramt
Speaker: Alice Weidel
p - Sehr geehrter Herr Präsident! Sehr geehrte Damen und Herren! Liebe Kollegen! Zum zweiten Mal in diesem Jahr beraten wir über den Entwurf eines Bundeshaushalts. Dazugelernt hat Ihre Regierung seither nichts, Frau Bundeskanzlerin.
kommentar - (Beifall bei der AfD – Johannes Kahrs [SPD]: Sie sollten sagen, wer Ihre Spender sind!)
p - Sie betreiben weiterhin eine Politik der Spaltung und der Unvernunft, eine Politik des Ausgabenwahns und der falschen Prioritäten, eine Politik, die den noch vorhandenen Wohlstand von heute bedenkenlos mit vollen Händen ausgibt und verschleudert, ohne an morgen zu denken.
kommentar - (Beifall bei der AfD)
p - Geändert hat sich aber die ökonomische Grundlage, auf der Sie diesen Ausverkauf betreiben. Das wirtschaftliche Fundament bröckelt. Ihre Politik ignoriert konsequent die ökonomische Vernunft. Sie setzt die Interessen des eigenen Landes und der eigenen Bürger a

p - Ist nicht von mir, das schrieb die „Neue Zürcher Zeitung“.
kommentar - (Beifall bei der AfD)
p - Entwicklungshilfeminister Gerd Müller sprach von 30 Milliarden Euro, die Bund, Länder und Gemeinden pro Jahr für 1 Million Migranten ausgeben. Da ungefähr 1,5 Millionen tatsächliche und angebliche Flüchtlinge seit 2015 zu uns gekommen sind, wären das 45 Milliarden Euro im Jahr. Das Institut der deutschen Wirtschaft kommt auf 50 Milliarden. Das Kieler Institut für Weltwirtschaft auf bis zu 55 Milliarden. Zum Vergleich – Haushalt! –: Die Ausgaben für Erziehungs- und Elterngeld des Bundes betrugen im vergangenen Jahr 6,4 Milliarden, für Bundesautobahnen und Bundesstraßen 6,6 Milliarden Euro. Der Jahresetat des Bundesforschungsministers – unsere Zukunft! – liegt bei 17 Milliarden Euro.
p - Aber, meine Damen und Herren, ist das Geld wenigstens gut angelegt? Ein Großteil der aktuellen Einwanderer gehört in die Kategorie Primär- oder Sekundäran­alphabeten.
kommentar - (Beifall bei der AfD)
p -

p - Auch an dieser Stelle höre ich wieder: Das geht alles nicht schnell genug. – Ich denke zurück: Ich kann mich nicht daran erinnern, dass der Vorgänger von Olaf Scholz jede Menge Gesetzesvorhaben eingebracht hat, um die Digitalisierung und die Besteuerung von Digitalkonzernen zu verbessern. Deswegen ist es richtig, meine Damen und Herren, dass wir hier gemeinsam mit unseren französischen Freunden vorangehen und dafür sorgen, dass Gewinne aus Digitalisierung auch in Deutschland und Europa besteuert werden; denn sonst fehlt am Ende das Geld für die notwendigen Investitionen.
kommentar - (Beifall bei Abgeordneten der SPD)
p - Sie sehen also: Diese Bundesregierung und die Koalition sind beim Thema Digitalisierung auf einem guten Weg. Das heißt nicht, dass man Dinge schönreden soll, das heißt nicht, dass man sich ausruhen darf. Aber: Wir stellen Milliarden für künstliche Intelligenz, für Infrastruktur und auch für den Kampf gegen negative Auswirkungen der Digitalisierung auf den Arbeitsma

p - Das Auswärtige Amt hat diese Einladung allen Fraktionen des Deutschen Bundestages zukommen lassen. Von der AfD ist meines Wissens kein einziger Abgeordneter bei einer dieser Konferenzen gewesen,
kommentar - (Udo Theodor Hemmelgarn [AfD]: Es war jemand da!)
p - und jetzt reißen Sie hier das Maul auf. Das ist unehrlich hoch drei!
kommentar - (Beifall bei der CDU/CSU, der SPD, der FDP, der LINKEN und dem BÜNDNIS 90/DIE GRÜNEN)
p - Ich entschuldige mich für das Wort „Maul“.
kommentar - (Dr. Bernd Baumann [AfD]: Heuchler!)
p - Die zweite große Herausforderung – der Bundesaußenminister hat davon gesprochen – bilden die transatlantische Zusammenarbeit und unsere Sicherheit. Ich möchte Sie ausdrücklich darin unterstützen, am 4. und 5. Dezember bei der Außenministerkonferenz der NATO die Chance zu ergreifen, den amerikanischen Präsidenten, die amerikanische Regierung davon abzubringen, den INF-Vertrag aufzukündigen. Ich glaube, wir sind uns bei der Bewertung, dass Russland den INF-Vertrag v

p - Für mich ist auch klar: Beschaffungen sind ureigene Aufgabe eines souveränen Staates. Das Zusammenspiel zwischen Industrie und Beschaffungsamt, zwischen Privat und Staat muss dahin gehend verbessert werden, dass der Prozess insgesamt schneller, effizienter und termintreuer wird – aber unter Federführung des Staates. Wir, die SPD-Fraktion, stellen uns klar gegen jegliche Privatisierungstendenzen, was ja auch schon betont wurde.
kommentar - (Beifall bei der SPD)
p - Es sind nicht das Amt und auch nicht der Charakter eines Amtes und schon gar nicht die dort Beschäftigten, die die Probleme verursachen, sondern es ist die Behäbigkeit des Prozesses an sich. Mir wäre es in diesem Zusammenhang auch viel lieber, wir würden die finanziellen Mittel nutzen und den Personalaufwuchs im BAAINBw oder in anderen nachgelagerten Behörden vorantreiben, als diese Mittel für externe Dritte zu verwenden.
kommentar - (Beifall bei Abgeordneten der SPD)
p - Der umständliche Beschaffungsprozess treibt mich u

kommentar - (Zuruf vom BÜNDNIS 90/DIE GRÜNEN: Unglaublich!)
p - In dieser Erklärung geht es um Menschenrechte und Lebensgrundlagen, um Selbstbestimmung, Arbeitsplätze und um die Zukunftschancen der Menschen. Damit geht es natürlich ganz zentral um Entwicklungsperspektiven.
p - Dieser Erklärung hätte die Regierung der Bundesrepublik Deutschland unbedingt zustimmen müssen. Aber unser Entwicklungsminister hat im Kabinett nicht dafür gekämpft, und er hat sich nicht mit dem Außenminister der SPD auseinandergesetzt. Da hätten Sie sich durchsetzen müssen.
kommentar - (Beifall beim BÜNDNIS 90/DIE GRÜNEN)
p - Traurig ist auch das Beispiel des Binding-Treaty-Prozesses. Das ist wirklich ein wichtiger entwicklungspolitischer Ansatz, der die Lieferkettenproblematik verbessert. Von Anfang an hat Ihr Kollege aus dem AA diesen Prozess torpediert. In der ersten Verhandlungsrunde – dafür schäme ich mich – wurde die Bundesrepublik Deutschland von einer Praktikantin vertreten, und am letzten Tag, in der v

---

In [9]:
# divs_to as tagesordnungspunkt 
speech_list = ['J','O','J_1']
i = 0
for div in divs_to: # to in tagesordnungspunkt list
    for uts in div.iter('rede'): 
        i += 1
        for c in uts.iter():
            if c.tag == "p" and c.get("klasse") in speech_list:
                print("{}: {}".format(c.tag, c.text))
                
            elif c.tag == "p" and c.get("klasse") == "redner":
                print("SPEAKER")
            elif c.tag == "kommentar":
                print("{}: {}".format(c.tag, c.text))
            print(i)

1
SPEAKER
1
1
OTHER SPEAKER
1
1
1
1
1
p: Sehr geehrter Herr Präsident! Sehr geehrte Damen und Herren! Liebe Kollegen! Zum zweiten Mal in diesem Jahr beraten wir über den Entwurf eines Bundeshaushalts. Dazugelernt hat Ihre Regierung seither nichts, Frau Bundeskanzlerin.
1
kommentar: (Beifall bei der AfD – Johannes Kahrs [SPD]: Sie sollten sagen, wer Ihre Spender sind!)
1
p: Sie betreiben weiterhin eine Politik der Spaltung und der Unvernunft, eine Politik des Ausgabenwahns und der falschen Prioritäten, eine Politik, die den noch vorhandenen Wohlstand von heute bedenkenlos mit vollen Händen ausgibt und verschleudert, ohne an morgen zu denken.
1
kommentar: (Beifall bei der AfD)
1
p: Geändert hat sich aber die ökonomische Grundlage, auf der Sie diesen Ausverkauf betreiben. Das wirtschaftliche Fundament bröckelt. Ihre Politik ignoriert konsequent die ökonomische Vernunft. Sie setzt die Interessen des eigenen Landes und der eigenen Bürger an die letzte Stelle. Dafür rennen Sie ideologischen 

12
p: Deutschland hat seine Vorreiterrolle bei Innovationen und Erfindungen lange eingebüßt. Die USA und Länder in Asien haben uns längst den Rang in Fragen der Digitalisierung, beim Ausbau von Breitbandnetzen und bei der Verkehrsinfrastruktur abgelaufen. Der Exportschlager Bildungssystem wurde in einer Folge von Experimenten geschwächt, unsere Universitäten europäisch egalisiert.
12
p: Die Europäische Union befindet sich durch die weiter schwelende Euro-Krise, wie wir alle wissen, nah am Rande eines Zusammenbruchs, und der Brexit zeigt, dass langfristig nicht Großbritannien, sondern vor allem deutsche und europäische Unternehmer das größte Interesse an normalen Wirtschaftsbeziehungen außerhalb der EU haben müssen.
12
p: Frau Merkel, Sie sind eine Kanzlerin der Isolation. Ihr Verzicht auf den Schutz deutscher Grenzen vor illegaler Migration und die damit verbundene Sogwirkung nach Afrika und in den Mittleren Osten haben die Axt an das freiheitliche Europa von Adenauer und de Gaulle gel

41
41
p: Herr Präsident! Verehrte Damen und Herren! Gerade ist dem Außenminister vorgeworfen worden, es gebe nicht genügend Initiativen aus Deutschland. Ich könnte die ganzen vier Minuten verbrauchen, um darüber zu reden. Ich will aber nur ganz kurz sagen: Fragen Sie mal den UN-Sondergesandten für Syrien, Herrn de Mistura, ob er sich unterstützt fühlt von Deutschland. Der Kollege Schraps und ich waren vor kurzem in der Ukraine. Fragen Sie dort mal nach, welche Rolle Deutschland im Normandie-Format spielt.
41
kommentar: (Dr. Franziska Brantner [BÜNDNIS 90/DIE GRÜNEN]: Das stimmt!)
41
p: Vor kurzem war Herr Krähenbühl von UNRWA da. Er hat uns noch mal berichtet, wie wichtig gerade die persönliche Initiative des Außenministers war, um die Finanzierung so weit sicherzustellen, wie sie sichergestellt werden konnte. Insofern: An Initiativen aus dem Auswärtigen Amt mangelt es wirklich nicht.
41
kommentar: (Beifall bei der SPD)
41
p: Liebe Kolleginnen und Kollegen, Deutschland ist in der Tat –

59
SPEAKER
59
59
OTHER SPEAKER
59
59
59
59
p: Sehr geschätzter Herr Präsident! Liebe Kolleginnen und Kollegen! Wir diskutieren heute, wie die verteidigungspolitischen Richtungsentscheidungen aussehen, und es ist spannend, festzustellen, aus welcher Ecke des Parlaments welche Kritik kommt. Die eine Ecke äußert, wir würden unüberlegt aufrüsten,
59
kommentar: (Niema Movassat [DIE LINKE]: Genau!)
59
p: die andere Ecke meint, wir müssten eher 70 Milliarden Euro für Verteidigung ausgeben, alles andere wäre verantwortungslos. Gut, dass sich die Mehrheit dieses Hauses an der Realität orientiert und einen ausgewogenen Haushalt vorlegt.
59
p: Mit den Schwerpunkten dieses Haushaltes reagieren wir auf die Veränderungen der Welt und stellen wir die Weichen für die kommenden Jahre. Wir kommen aus einer Welt, in der Begriffe wie „dynamisches Verfügbarkeitsmanagement“ ein schickes Label für den verwalteten Mangel waren. Wir kommen aus einer Welt, Ende der 80er-, Anfang der 90er-Jahre, in der Landes- u

In [9]:
# divs as rede 
speech_list = ['J','O','J_1']
i = 0
for uts in divs: # p in rede
    i += 1
    for c in uts.iter(): # iterate over p 
        if c.tag == "p" and c.get("klasse") in speech_list:
            print("{}: {}".format(c.tag, c.text))
        elif c.tag == "p" and c.get("klasse") == "redner":
            print("{}: REDNER".format(c.tag, c.text))
        elif c.tag == "kommentar":
            print("{}: {}".format(c.tag, c.text))
        print(i)

1
p: REDNER
1
1
1
1
1
1
1
1
p: Guten Morgen, liebe Kolleginnen und Kollegen! Nehmen Sie bitte Platz.
1
p: Meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen! Ich begrüße Sie zur konstituierenden Sitzung des 19. Deutschen Bundestages. Es entspricht der ständigen Übung, zu Beginn der konstituierenden Sitzung nach den Regelungen der bisherigen Geschäftsordnung des Deutschen Bundestages zu verfahren.
1
p: § 1 Absatz 2 der Geschäftsordnung des Deutschen Bundestages sieht vor, dass das am längsten dem Bundestag angehörende Mitglied, das hierzu bereit ist, den Vorsitz übernimmt, bis der Deutsche Bundestag einen Präsidenten gewählt hat.
1
p: Die Fraktion der AfD widerspricht diesem Verfahren und hat auf Drucksache 19/2 beantragt, einen Versammlungsleiter zu wählen, der die konstituierende Sitzung eröffnen soll. Über diesen Antrag lasse ich sofort abstimmen. Wer dem Antrag der Fraktion der AfD zustimmt, den bitte ich um sein Handzeichen. – Gegenstimmen? –
1
kommentar: (Lachen

In [41]:
# removing repeating utterances
# speeches after first <name> in each <rede> not showing 
for div in divs:
    for uts in div.iter():
        # first part of speech
        for c in uts.findall('p'):
            if c.get("klasse") == "redner":
                if c.xpath('following-sibling::p[count(preceding-sibling::name) = 0]'):
                    print(c.xpath('descendant::nachname/text()')[0])
                    for j in c.xpath('following-sibling::p[count(preceding-sibling::name) = 0]'):
                        print("[M]", j.tag, "-", j.text)
                    print("----END----")
        
        # procedural speech in between 
        for sp in uts.findall('name'):
            if sp.text:
                print("[N]", sp.tag, sp.text)
                for k in sp.xpath('following-sibling::p'):
                    print("[N1]", k.tag, "-", k.text)
                print("----END----")
            

Weidel
[M] p - Sehr geehrter Herr Präsident! Sehr geehrte Damen und Herren! Liebe Kollegen! Zum zweiten Mal in diesem Jahr beraten wir über den Entwurf eines Bundeshaushalts. Dazugelernt hat Ihre Regierung seither nichts, Frau Bundeskanzlerin.
[M] p - Sie betreiben weiterhin eine Politik der Spaltung und der Unvernunft, eine Politik des Ausgabenwahns und der falschen Prioritäten, eine Politik, die den noch vorhandenen Wohlstand von heute bedenkenlos mit vollen Händen ausgibt und verschleudert, ohne an morgen zu denken.
[M] p - Geändert hat sich aber die ökonomische Grundlage, auf der Sie diesen Ausverkauf betreiben. Das wirtschaftliche Fundament bröckelt. Ihre Politik ignoriert konsequent die ökonomische Vernunft. Sie setzt die Interessen des eigenen Landes und der eigenen Bürger an die letzte Stelle. Dafür rennen Sie ideologischen Weltbeglückungsfantasien hinterher, und das hält auch die stärkste Volkswirtschaft auf Dauer nicht ohne Schaden aus.
[M] p - Die Automobilindustrie, ein Eck

[M] p - Darum braucht es eine Klimaabgabe, um unsere Wirtschaft vor Klimadumping zu schützen, um faire Wettbewerbsbedingungen für alle herzustellen. Dafür brauchen wir eine starke Europäische Union; denn eine starke Europäische Union ist die Antwort auf die Frage des Stahlarbeiters. Nur mit der Europäischen Union können wir Klima- und Sozialdumping etwas entgegensetzen, und mit einer starken Europäischen Union können wir die Klimaziele erreichen.
[M] p - Aber leider ist diese Bundesregierung zum Bremser und Blockierer geworden, wo beherzte Maßnahmen notwendig wären. Wohin das führt, sehen wir doch an den aktuellen Wirtschaftszahlen: Wir erleben das erste Mal seit Jahren wieder ein Minuswachstum, vor allem wegen der schlechten Zahlen in der Autoindustrie – weil die Autobosse betrogen und den Umstieg auf abgasarme Autos verschlafen und auch verschleppt haben.
[M] p - Auch Sie, liebe Bundesregierung, scheitern daran, für den Aufbruch, für eine zukunftsfähige Autoindustrie zu sorgen, um la

[N1] p - Vielen Dank. – Nächste Rednerin ist für die Fraktion Bündnis 90/Die Grünen Tabea Rößner.
----END----
Rößner
[M] p - Herr Präsident! Meine Damen und Herren! Frau Bundeskanzlerin, Sie haben Ihre Digitalklausur vergangene Woche angesprochen. Ganz stolz haben Sie da eine Umsetzungsstrategie zur Digitalisierung präsentiert. Aber in den Medien gab es keine Begeisterungsstürme. Warum nicht? Ich sage Ihnen, warum. Bei dieser Strategie fehlt Ihnen nämlich der Blick fürs Ganze. Doch dieser wäre dringend notwendig.
[M] p - Die Umsetzungsstrategie ist nicht mehr als ein Sammelsurium an Willensbekundungen. Aber erstens kann man sich davon nichts kaufen; ich denke da an die leidvolle Geschichte des Breitbandausbaus. Auch jetzt ist das Kapitel zum wichtigsten Infrastrukturprojekt äußerst ­schmal – wie auch das Budget. Und zweitens fehlt einfach die übergeordnete Vision. Sie müssten viel konsequenter von der Zukunft her denken und die Strategie danach ausrichten. Aber genau das fehlt. Und das

[M] p - Vielen herzlichen Dank.
----END----
[N] name Vizepräsident Dr. Hans-Peter Friedrich:
[N1] p - Der nächste Redner ist der Kollege Dr. Norbert Röttgen, CDU/CSU-Fraktion.
----END----
Röttgen
[M] p - Sehr geehrter Herr Präsident! Liebe Kolleginnen und Kollegen! Die Bundeskanzlerin hat heute Morgen in ihrer Rede gesagt, wir hätten eine manchmal auch sehr nervöse Zeit. Das halte ich für eine bemerkenswerte Beschreibung
[M] p - sozusagen der psychischen Verfassung der Welt. Der Bundesaußenminister hat für Deutschland die Frage gestellt: Was liegt eigentlich noch in unserer Gestaltungsmacht? Die Bundeskanzlerin hat übrigens ihrer Feststellung den Hinweis folgen lassen, dass wir uns die Frage stellen müssen: Was ist unser Beitrag in dieser nervösen Zeit? Ich möchte mich in meinen sechs Minuten mit dieser Frage beschäftigen: Was kann, was soll unser deutscher Beitrag in dieser nervösen Zeit sein?
[M] p - Der Außenminister hat die Frage beantwortet und gesagt: Europa ist die Antwort auf d

[N] name Vizepräsident Thomas Oppermann:
[N1] p - Vielen Dank. – Letzte Rednerin in dieser Debatte ist die Kollegin Anita Schäfer für die Fraktion CDU/CSU.
----END----
Schäfer
[M] p - Sehr verehrter Herr Präsident! Liebe Frau Ministerin! Sehr verehrte Kolleginnen und Kollegen! Nun, da wir über den Haushalt des Einzelplans 14 für das Jahr 2019 debattieren, möchte ich einige sicherheitspolitische Aspekte hervorheben. Am Wochenende war ich auf der 64. Jahrestagung der Parlamentarischen Versammlung der NATO in Kanada. Immer wieder wurde ich von Abgeordneten verbündeter Staaten auf den Zustand der Bundeswehr angesprochen. Dass gerade das reiche Deutschland vom 2-Prozent-Ziel weit entfernt ist, war natürlich auch ein Thema.
[M] p - Unsere Partner machen sich Gedanken über die Rolle Deutschlands im Bündnis. Und wir, meine Damen und Herren, müssen das auch tun.
[M] p - Bis heute tragen die USA die Hauptlast der Kosten für die Sicherheit der NATO und damit auch für unsere Sicherheit. Für CDU un

[N] name Vizepräsident Wolfgang Kubicki:
[N1] p - Herr Kollege, kommen Sie bitte zum Schluss.
[N1] p - None
[N1] p - Gleich.
[N1] p - Herr Minister, hören Sie auf, ein Strategiepapier nach dem anderen zu produzieren! Kein Mensch braucht diese nichtssagenden, schon x-mal veröffentlichten Statements. Sie haben es doch richtig erkannt und auch richtig formuliert.
[N1] p - Herr Kollege, Sie haben jetzt noch einen Satz.
[N1] p - None
[N1] p - Ich bin in zwei Sekunden fertig. – Wir haben keinen Mangel an unnötigen Veröffentlichungen, sondern Handlungsprobleme. Das sind die Probleme.
[N1] p - Vielen Dank, Herr Kollege Kekeritz. – Als Nächstes spricht zu uns der Kollege Hermann Gröhe, CDU/CSU-Fraktion.
----END----
[N] name Vizepräsident Wolfgang Kubicki:
[N1] p - Herr Kollege, Sie haben jetzt noch einen Satz.
[N1] p - None
[N1] p - Ich bin in zwei Sekunden fertig. – Wir haben keinen Mangel an unnötigen Veröffentlichungen, sondern Handlungsprobleme. Das sind die Probleme.
[N1] p - Vielen Dank, 

---

### Speaker

In [46]:
count = 0
for div in divs: # <rede> in [all <rede>]
    count += 1
    for uts in div: # <p> in <rede>
        if uts.get("klasse") == "redner": # <p>
            for redner in uts: # <redner>
                vorname = redner.xpath('name/vorname/text()')
                nachname = redner.xpath('name/nachname/text()')
                fullname = vorname + nachname
                if len(fullname) > 1:
                    fullname = fullname[0] + ' ' + fullname[1]
                if fullname != []:
                    print(count, fullname)

1 Alice Weidel
1 Alice Weidel
2 Angela Merkel
2 Angela Merkel
2 Angela Merkel
3 Christian Lindner
4 Andrea Nahles
5 Sahra Wagenknecht
6 Anton Hofreiter
7 Ralph Brinkhaus
8 Alexander Gauland
9 Achim Post
10 Otto Fricke
11 Alexander Dobrindt
12 Frauke Petry
13 Johannes Kahrs
14 Patricia Lips
15 Sonja Amalie Steffen
15 Sonja Amalie Steffen
16 Harald Weyel
16 Harald Weyel
17 Sonja Amalie Steffen
18 Norbert Kleinwächter
19 Gitta Connemann
20 Hartmut Ebbing
21 Katrin Budde
22 Doris Achelwilm
23 Tabea Rößner
24 Jens Zimmermann
25 Erhard Grundl
25 Erhard Grundl
26 Birgit Malsack-Winkemann
27 Doris Barnett
28 Bijan Djir-Sarai
29 Alois Karl
30 Gregor Gysi
31 Ekin Deligöz
32 Heiko Maas
32 Heiko Maas
33 Heike Hänsel
34 Frank Müller-Rosentritt
35 Heiko Maas
36 Armin-Paulus Hampel
36 Armin-Paulus Hampel
37 Jürgen Hardt
37 Jürgen Hardt
37 Armin-Paulus Hampel
37 Jürgen Hardt
37 Jürgen Hardt
37 Alexander S. Neu
37 Jürgen Hardt
38 Michael Georg Link
39 Heike Hänsel
40 Franziska Brantner
41 Frank Schwabe

In [29]:
for div in divs_to: 
    for uts in div.iter('rede'): # <p> in <rede>
        for redner in uts.xpath("p[@klasse='redner']/redner"):
            vorname = redner.xpath('name/vorname/text()')
            nachname = redner.xpath('name/nachname/text()')
            fullname = vorname + nachname
            if len(fullname) > 1:
                fullname = fullname[0] + ' ' + fullname[1]
            if fullname != []:
                print(fullname)

Alice Weidel
Alice Weidel
Angela Merkel
Angela Merkel
Angela Merkel
Christian Lindner
Andrea Nahles
Sahra Wagenknecht
Anton Hofreiter
Ralph Brinkhaus
Alexander Gauland
Achim Post
Otto Fricke
Alexander Dobrindt
Frauke Petry
Johannes Kahrs
Patricia Lips
Sonja Amalie Steffen
Sonja Amalie Steffen
Harald Weyel
Harald Weyel
Sonja Amalie Steffen
Norbert Kleinwächter
Gitta Connemann
Hartmut Ebbing
Katrin Budde
Doris Achelwilm
Tabea Rößner
Jens Zimmermann
Erhard Grundl
Erhard Grundl
Birgit Malsack-Winkemann
Doris Barnett
Bijan Djir-Sarai
Alois Karl
Gregor Gysi
Ekin Deligöz
Heiko Maas
Heiko Maas
Heike Hänsel
Frank Müller-Rosentritt
Heiko Maas
Armin-Paulus Hampel
Armin-Paulus Hampel
Jürgen Hardt
Jürgen Hardt
Armin-Paulus Hampel
Jürgen Hardt
Jürgen Hardt
Alexander S. Neu
Jürgen Hardt
Michael Georg Link
Heike Hänsel
Franziska Brantner
Frank Schwabe
Norbert Röttgen
Matern von Marschall
Frank Heinrich
Martin Hohmann
Reinhard Brandl
Karsten Klein
Dennis Rohde
Gesine Lötzsch
Tobias Lindner
Ursula Leyen

In [30]:
from parsing_utils import find_person_in_db, clean_text
from regular_expressions_global import *

# test speaker name processing 
test_name = "Präsident Dr. Wolfgang Schäuble:"

position = PERSON_POSITION.search(test_name)
cname = PERSON_POSITION.sub('', test_name).strip(' ,')

title = TITLE.match(cname)
cname = TITLE.sub('', cname).strip()

cname

'Wolfgang Schäuble:'

### Speaker Metadata

info needed for find_person_in_db.py: <br>
name, position (rolle_lang), party (fraktion), wp (wahlperiode), session (sitzungnr) <br>
name_id with redner_id ? [might be difficult if speaker has already been added before]

## Building a Scraper

In [1]:
import sys
import os

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.core import serializers

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


### import serialized data for parliamentarians 

In [18]:
with open("german_parties.xml", "r") as data:
    for obj in serializers.deserialize("xml", data):
        obj.save()

In [19]:
with open("german_parliamentarians.xml", "r") as data:
    for obj in serializers.deserialize("xml", data):
        obj.save()

In [26]:
cmodels.Country.objects.get(name="Germany").id

2921044

In [27]:
# check for parliamentarians
pm.Person.objects.filter(active_country=cmodels.Country.objects.get(name="Germany").id)

<QuerySet [<Person: Dr. Manfred Abelein>, <Person: Dr. Ernst Achenbach>, <Person: Annemarie Ackermann>, <Person: Dr. Else Ackermann>, <Person: Ulrich Adam>, <Person: Rudolf Adams>, <Person: Raban Adelmann>, <Person: Dr. Konrad Adenauer>, <Person: Brigitte Adler>, <Person: Eduard Adorno>, <Person: Dr. Jochen Aerssen>, <Person: Christa Nikkels>, <Person: Willi Agatz>, <Person: Conrad Ahlers>, <Person: Adolf Ahrens>, <Person: Hermann Ahrens (Salzgitter)>, <Person: Dr. Karl Ahrens>, <Person: Dr. Heinrich Aigner>, <Person: Siegbert Alber>, <Person: Johannes Albers>, '...(remaining elements truncated)...']>

In [19]:
pm.Person.objects.filter(surname="Gauland")[0].id

68692

In [3]:
pm.Person.objects.filter(surname="Schulze")

<QuerySet [<Person: Svenja Schulze>, <Person: Gerhard Schulze (Berlin)>, <Person: Waldemar Schulze (Berlin)>, <Person: Frederick Schulze (Sangerhausen)>, <Person: Dr. Klaus-Peter Schulze>, <Person: Frederik Schulze>, <Person: Schulze>]>

In [21]:
pm.Person.objects.get(id=68692).party

In [33]:
pm.Person.objects.filter(positions="{Alterspräsident}")

<QuerySet []>

In [39]:
pm.Person.objects.filter(surname="Merkel")[0].positions

['parliamentarian']

## Inspecting scraper results

In [56]:
import numpy as np

In [7]:
# Deleting bad scraper runs
pm.Document.objects.filter(id=195).delete()

(1864,
 {'parliament.Document_search_matches': 0,
  'scoping.Note': 0,
  'scoping.DocOwnership': 0,
  'tmv_app.DocTopic': 0,
  'parliament.Utterance_search_matches': 0,
  'parliament.Paragraph_search_matches': 0,
  'parliament.Interjection_parties': 390,
  'parliament.Interjection_persons': 228,
  'parliament.Interjection': 475,
  'parliament.Paragraph': 457,
  'parliament.Utterance': 313,
  'parliament.Document': 1})

In [5]:
pm.Utterance.objects.filter(document__id=6423)

<QuerySet [<Utterance: Utterance object (1807547)>, <Utterance: Utterance object (1807546)>, <Utterance: Utterance object (1807545)>, <Utterance: Utterance object (1807544)>, <Utterance: Utterance object (1807543)>, <Utterance: Utterance object (1807542)>, <Utterance: Utterance object (1807541)>, <Utterance: Utterance object (1807540)>, <Utterance: Utterance object (1807539)>, <Utterance: Utterance object (1807538)>, <Utterance: Utterance object (1807537)>, <Utterance: Utterance object (1807536)>, <Utterance: Utterance object (1807535)>, <Utterance: Utterance object (1807534)>, <Utterance: Utterance object (1807533)>, <Utterance: Utterance object (1807532)>, <Utterance: Utterance object (1807531)>, <Utterance: Utterance object (1807530)>, <Utterance: Utterance object (1807529)>]>

In [11]:
pm.Paragraph.objects.filter(utterance__id=1807535)[3].text

'Auch Die Linke hat ihre bewährte Rolle. Sie gibt hier die Scheinheilige. Herr Korte trägt vor, es ginge darum, das Parlament schnell arbeitsfähig zu machen. Sie kennen die Praxis des Parlamentarismus in Deutschland ganz genau. Wir wissen heute nicht, welche Ausschüsse es insgesamt geben wird, wie groß sie sein werden, welche Kollegen wohin gehen müssen. Dass das in den Fraktionen natürlich besprochen werden muss, ist Ihnen bekannt.'

In [8]:
pm.Document.objects.last().id

187

In [14]:
pm.Document.objects.get(id=6423).parlperiod

<ParlPeriod: Germany - National - 19>

In [5]:
pm.Document.objects.filter(parlperiod__n=12)

<QuerySet [<Document: Plenarprotokoll, 12/13, 1991-03-12>, <Document: Plenarprotokoll, 12/12, 1991-02-28>]>